# 🎯 Titanic Survival Prediction - Hyperparameter Tuning

## 📊 Mục tiêu
- Tối ưu hóa hyperparameters cho các models tốt nhất
- Sử dụng GridSearchCV và RandomSearchCV
- So sánh performance trước và sau tuning
- Chọn best parameters cho production

## 📋 Nội dung
1. **Data Preparation**
2. **Random Forest Tuning**
3. **XGBoost Tuning**
4. **Logistic Regression Tuning**
5. **SVM Tuning**
6. **Model Comparison After Tuning**
7. **Best Model Selection**


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import joblib
import time
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries imported successfully!")
print("🎨 Visualization style set!")
